Nombre: Cristina Molina

# Instalación de Bibliotecas

In [1]:
!pip install nltk

In [2]:
!pip install --upgrade numpy

# Prepocesamiento de la Data

En esta sección, llevamos a cabo el preprocesamiento del texto contenido en archivos de texto plano. Este preprocesamiento implica una serie de pasos para limpiar y estructurar el texto de manera que sea más adecuado para su posterior análisis. Los pasos específicos incluyen:

1. **Lectura de Archivos**: Utilizamos la función `preprocess_text` para leer el contenido de un archivo dado y cargarlo en una variable de texto.

2. **Conversión a Minúsculas**: Para asegurarnos de que no haya ambigüedades en el análisis, convertimos todo el texto a minúsculas usando `text.lower()`.

3. **Tokenización**: Dividimos el texto en palabras o "tokens" utilizando expresiones regulares. La función `re.findall(r'\b\w+\b', text)` busca todas las secuencias de caracteres alfanuméricos y las trata como tokens. Esto nos permite trabajar con unidades significativas de texto en lugar de caracteres individuales.

4. **Eliminación de Duplicados**: En algunos casos, un mismo token puede aparecer múltiples veces en un archivo o en varios archivos. Para evitar redundancias y mejorar la eficiencia del análisis, eliminamos los duplicados de la lista de tokens utilizando `list(dict.fromkeys(tokens))`.

5. **Aplicación de Stemming**: Finalmente, aplicamos stemming a cada token para reducir las palabras a su forma raíz o "stem". El stemmer utilizado en este caso es el SnowballStemmer del paquete NLTK. Esto nos permite tratar palabras con la misma raíz como si fueran la misma palabra, lo que simplifica el análisis y la búsqueda.

El resultado de este proceso es una lista de tokens procesados y normalizados que están listos para ser utilizados en la construcción del índice invertido y otros análisis posteriores.

Este preprocesamiento es fundamental en el procesamiento de lenguaje natural, ya que ayuda a reducir la complejidad del texto y a extraer las características más relevantes para su análisis.
a su análisis.

In [3]:
import os
import re
from nltk.stem import SnowballStemmer

def preprocess_text(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        text = file.read()

    # Convertir el texto a minúsculas
    text = text.lower()

    # Dividir el texto en tokens (palabras)
    tokens = re.findall(r'\b\w+\b', text)

    # Eliminar duplicados de la lista de tokens
    tokens = list(dict.fromkeys(tokens))

    # Inicializar el stemmer
    stemmer = SnowballStemmer('english')

    # Aplicar stemming a cada token
    stemmed_tokens = [stemmer.stem(token) for token in tokens]

    return stemmed_tokens

# Directorio donde se encuentran los archivos TXT
directory = 'Data'

# Lista para almacenar los tokens procesados de todos los archivos
all_tokens = []

# Procesar cada archivo en el directorio
for filename in os.listdir(directory):
    if filename.endswith('.txt'):
        file_path = os.path.join(directory, filename)
        tokens = preprocess_text(file_path)
        all_tokens.extend(tokens)

# Eliminar duplicados de la lista de tokens de todos los archivos
all_tokens = list(dict.fromkeys(all_tokens))

# Creación del Índice Invertido


En esta sección, construimos el índice invertido a partir de los tokens procesados de los archivos de texto. El índice invertido es una estructura de datos que asigna cada término (o token) a la lista de documentos en los que aparece. Los pasos específicos para construir el índice invertido incluyen:

1. **Lectura de Archivos**: Iteramos sobre todos los archivos de texto en el directorio especificado y extraemos los tokens procesados de cada archivo utilizando la función `preprocess_text`.

2. **Construcción del Índice**: Para cada token procesado, verificamos si ya está presente en el índice invertido. Si no está presente, lo añadimos como una nueva entrada en el índice con un conjunto vacío de documentos asociados. Luego, agregamos el nombre del archivo actual al conjunto de documentos asociados con el token.

3. **Creación del DataFrame**: Utilizamos la estructura de datos DataFrame de la biblioteca pandas para representar el índice invertido. Creamos un DataFrame donde las filas representan los términos y las columnas representan los nombres de los archivos. Inicializamos todas las celdas del DataFrame con ceros.

4. **Asignación de Valores**: Recorremos el índice invertido y asignamos un valor de 1 a las celdas correspondientes en el DataFrame donde el término aparece en un archivo específico. Esto nos da una representación binaria del índice invertido, donde un valor de 1 indica la presencia del término en un archivo y un valor de 0 indica lo contrario.

5. **Guardado en Archivo CSV**: Finalmente, guardamos el DataFrame del índice invertido en un archivo CSV para su posterior uso y análisis.

El resultado es un índice invertido completo que mapea cada término a los documentos en los que aparece, proporcionando una forma eficiente de buscar y recuperar información relevante a partir de los términos de búsqueda.


In [4]:
import os
import re
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer

def construct_inverted_index(directory):
    inverted_index = {}
    files = os.listdir(directory)
    
    for file in files:
        file_path = os.path.join(directory, file)
        tokens = preprocess_text(file_path)
        
        for token in tokens:  
            if token not in inverted_index:
                inverted_index[token] = set()
            inverted_index[token].add(file)
    
    df = pd.DataFrame(index=inverted_index.keys(), columns=files)
    df.fillna(0, inplace=True)
    
    for token, files in inverted_index.items():
        for file in files:
            df.at[token, file] = 1
    
    df.to_csv('inverted_index.csv')
    
    return df

directory = 'Data'

indice_invertido_df = construct_inverted_index(directory)  # Pasa el argumento 'directory' aquí

print("DataFrame del Índice Invertido:")
print(indice_invertido_df)

DataFrame del Índice Invertido:
               pg100.txt  pg10676.txt  pg1080.txt  pg10907.txt  pg11.txt  \
the                    1            1           1            1         1   
project                1            1           1            1         1   
gutenberg              1            1           1            1         1   
ebook                  1            1           1            1         1   
of                     1            1           1            1         1   
...                  ...          ...         ...          ...       ...   
quixano                0            0           0            0         0   
quixana                0            0           0            0         0   
p74b                   0            0           0            0         0   
tordesillesqu          0            0           0            0         0   
p74e                   0            0           0            0         0   

               pg1184.txt  pg120.txt  pg1232.txt  pg125

# Ejemplo de Uso

In [5]:
import pandas as pd

# Cargar el índice invertido desde el archivo CSV
indice_invertido_df = pd.read_csv('inverted_index.csv', index_col=0)

def buscar_palabra_en_indice(palabra):
    palabra = palabra.lower()  # Convertir la palabra a minúsculas para que coincida con el formato en el DataFrame
    if palabra in indice_invertido_df.index:
        pd.set_option('display.max_rows', None)  # Mostrar todas las filas
        pd.set_option('display.max_columns', None)  # Mostrar todas las columnas
        resultados = indice_invertido_df.loc[palabra]
        print("Resultados para la palabra '{}':".format(palabra))
        print(resultados[resultados > 0])  # Mostrar solo los libros donde la palabra aparece al menos una vez
    else:
        print("La palabra '{}' no se encuentra en ningún libro.".format(palabra))

# Ejemplo de uso:
buscar_palabra_en_indice("protectress")


Resultados para la palabra 'protectress':
pg100.txt      1
pg2600.txt     1
pg41445.txt    1
pg59468.txt    1
pg84.txt       1
Name: protectress, dtype: int64


# Implementing Boolean Search

### Descripción
Esta función realiza una búsqueda booleana en un índice invertido representado por un DataFrame de pandas. Permite realizar consultas con los operadores booleanos AND, OR y NOT.

### Parámetros
- `query` (str): La consulta booleana que se va a realizar.
- `inverted_index_df` (DataFrame): El DataFrame que contiene el índice invertido.

### Returns
- result (set): Un conjunto de nombres de archivos que cumplen con la consulta booleana.

### Comportamiento
La función divide la consulta en partes utilizando el método split(). Luego, utiliza un bucle while para iterar sobre cada parte de la consulta.
- Si la parte actual es 'AND', realiza una intersección entre los resultados de las operaciones anteriores y los resultados de la operación actual.
- Si la parte actual es 'OR', realiza una unión entre los resultados de las operaciones anteriores y los resultados de la operación actual.
- Si la parte actual es 'NOT', realiza una diferencia entre los resultados de las operaciones anteriores y los resultados de la operación actual.
- Si la parte actual no es un operador booleano, busca en el índice invertido y devuelve los archivos asociados a esa palabra.

Finalmente, la función devuelve un conjunto de nombres de archivos que cumplen con la consulta booleana.
### Ejemplo de Uso

In [6]:
def buscar_con_booleanos(query, inverted_index_df):
    pd.set_option('display.max_rows', None)  # Mostrar todas las filas
    pd.set_option('display.max_columns', None)  # Mostrar todas las columnas

    query_parts = query.split()
    result = None
    
    while query_parts:
        token = query_parts.pop(0)
        if token == 'AND':
            operand1 = result
            operand2 = set()
            next_token = query_parts.pop(0)
            if next_token in inverted_index_df.index:
                operand2 = set(inverted_index_df.loc[next_token][inverted_index_df.loc[next_token] > 0].index)
            result = operand1.intersection(operand2)
        elif token == 'OR':
            operand1 = result
            operand2 = set()
            next_token = query_parts.pop(0)
            if next_token in inverted_index_df.index:
                operand2 = set(inverted_index_df.loc[next_token][inverted_index_df.loc[next_token] > 0].index)
            result = operand1.union(operand2)
        elif token == 'NOT':
            operand1 = result
            operand2 = set()
            next_token = query_parts.pop(0)
            if next_token in inverted_index_df.index:
                operand2 = set(inverted_index_df.loc[next_token][inverted_index_df.loc[next_token] > 0].index)
            result = operand1.difference(operand2)
        else:
            result = set()
            if token in inverted_index_df.index:
                result = set(inverted_index_df.loc[token][inverted_index_df.loc[token] > 0].index)
    
    if result:
        print("Documentos que cumplen con la consulta:")
        print(result)
    else:
        print("No se encontraron documentos que cumplan con la consulta.")

# Ejemplo de uso:
query = "william AND shakespear OR this"
buscar_con_booleanos(query, indice_invertido_df)

Documentos que cumplen con la consulta:
{'pg43.txt', 'pg1259.txt', 'pg2814.txt', 'pg5197.txt', 'pg98.txt', 'pg52882.txt', 'pg10907.txt', 'pg1998.txt', 'pg18893.txt', 'pg67098.txt', 'pg55.txt', 'pg1232.txt', 'pg64317.txt', 'pg46.txt', 'pg16389.txt', 'pg16.txt', 'pg47312.txt', 'pg45540.txt', 'pg2641.txt', 'pg2160.txt', 'pg52281.txt', 'pg145.txt', 'pg1400.txt', 'pg2542.txt', 'pg7370.txt', 'pg47629.txt', 'pg1342.txt', 'pg26073.txt', 'pg20228.txt', 'pg6761.txt', 'pg67979.txt', 'pg41070.txt', 'pg408.txt', 'pg42933.txt', 'pg27827.txt', 'pg11.txt', 'pg41445.txt', 'pg768.txt', 'pg6130.txt', 'pg996.txt', 'pg2554.txt', 'pg44388.txt', 'pg21012.txt', 'pg45.txt', 'pg205.txt', 'pg514.txt', 'pg73444.txt', 'pg25344.txt', 'pg345.txt', 'pg844.txt', 'pg37106.txt', 'pg45848.txt', 'pg30254.txt', 'pg73447.txt', 'pg28054.txt', 'pg4300.txt', 'pg29728.txt', 'pg44837.txt', 'pg61419.txt', 'pg219.txt', 'pg1513.txt', 'pg2600.txt', 'pg50038.txt', 'pg2591.txt', 'pg1080.txt', 'pg244.txt', 'pg59469.txt', 'pg35899.txt',

# Interfaz de Usuario


En esta sección, implementamos una interfaz de usuario web para permitir a los usuarios realizar búsquedas en el índice invertido construido previamente. Los pasos específicos para la interfaz de usuario incluyen:

1. **Instalación de Bibliotecas**: Utilizamos pip para instalar las bibliotecas necesarias, incluyendo Flask para el desarrollo del servidor web y Waitress para el servidor de producción.

2. **Definición de Rutas**: Configuramos rutas en la aplicación Flask para manejar las solicitudes del cliente. La ruta principal carga la página HTML que contiene el formulario de búsqueda, mientras que la ruta `/search` procesa la solicitud de búsqueda y devuelve los resultados.

3. **Carga del Índice Invertido**: Cargamos el índice invertido desde el archivo CSV generado previamente. Utilizamos pandas para cargar el índice invertido en un DataFrame para facilitar la manipulación y búsqueda de datos.

4. **Página HTML**: Creamos una página HTML simple que contiene un formulario de búsqueda con un campo de entrada para que el usuario ingrese la palabra clave de búsqueda. Este formulario envía la palabra clave al servidor Flask cuando se envía.

5. **Procesamiento de la Búsqueda**: En la ruta `/search`, recibimos la palabra clave ingresada por el usuario y verificamos si está presente en el índice invertido. Si la palabra clave está presente, devolvemos los archivos en los que aparece la palabra clave como resultado de la búsqueda. Si no está presente, mostramos un mensaje indicando que la palabra clave no está en el índice invertido.

6. **Servidor en Ejecución**: Ejecutamos el servidor Flask para que esté disponible en una dirección IP y puerto específicos. Utilizamos Waitress para servir la aplicación Flask en un entorno de producción.

Con esta interfaz de usuario, los usuarios pueden realizar búsquedas simples en el índice invertido y obtener resultados relevantes en función de las palabras clave ingresadas.


In [7]:
!pip install flask
!pip install gunicorn
!pip install waitress

In [8]:
%%writefile wsgi.py
from busqueda import app as application

if __name__ == "__main__":
    application.run()

Overwriting wsgi.py


In [9]:
from flask import Flask, render_template, request, jsonify
from waitress import serve
import pandas as pd

app = Flask(__name__)

# Cargar el índice invertido desde el archivo CSV
indice_invertido_df = pd.read_csv('inverted_index.csv', index_col=0)

# Ruta principal que carga la página HTML
@app.route('/')
def index():
    return render_template('index.html')

# Ruta para realizar la búsqueda de palabras en el índice invertido
@app.route('/search')
def search():
    keyword = request.args.get('keyword')

    # Verificar si la palabra está en el índice invertido
    keyword = keyword.lower()  # Convertir la palabra a minúsculas para que coincida con el formato en el DataFrame
    if keyword in indice_invertido_df.index:
        # Obtener los archivos en los que aparece la palabra
        files = indice_invertido_df.loc[keyword][indice_invertido_df.loc[keyword] > 0].index.tolist()
        return jsonify({'files': files})
    else:
        return jsonify({'message': 'La palabra no está en el índice invertido'})

# Ruta para realizar la búsqueda booleana
@app.route('/boolean_search')
def boolean_search():
    query = request.args.get('query')
    result = buscar_con_booleanos(query, indice_invertido_df)
    return jsonify({'resultados': list(result)})

# Función para buscar con booleanos
def buscar_con_booleanos(query, inverted_index_df):
    query_parts = query.split()
    result = None
    
    while query_parts:
        token = query_parts.pop(0)
        if token == 'AND':
            operand1 = result
            operand2 = set()
            next_token = query_parts.pop(0)
            if next_token in inverted_index_df.index:
                operand2 = set(inverted_index_df.loc[next_token][inverted_index_df.loc[next_token] > 0].index)
            result = operand1.intersection(operand2)
        elif token == 'OR':
            operand1 = result
            operand2 = set()
            next_token = query_parts.pop(0)
            if next_token in inverted_index_df.index:
                operand2 = set(inverted_index_df.loc[next_token][inverted_index_df.loc[next_token] > 0].index)
            result = operand1.union(operand2)
        elif token == 'NOT':
            operand1 = result
            operand2 = set()
            next_token = query_parts.pop(0)
            if next_token in inverted_index_df.index:
                operand2 = set(inverted_index_df.loc[next_token][inverted_index_df.loc[next_token] > 0].index)
            result = operand1.difference(operand2)
        else:
            result = set()
            if token in inverted_index_df.index:
                result = set(inverted_index_df.loc[token][inverted_index_df.loc[token] > 0].index)
    
    return result

if __name__ == "__main__":
    host = '127.0.0.1'
    port = 5000
    print(f"Servidor en ejecución en http://{host}:{port}")
    serve(app, host=host, port=port)

Servidor en ejecución en http://127.0.0.1:5000
